In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
from tqdm import tqdm
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
%matplotlib inline
test_path = 'https://media.githubusercontent.com/media/cchopade/gahack2/master/test.csv'
train_path = 'https://media.githubusercontent.com/media/cchopade/gahack2/master/train.csv'

In [130]:
#df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
#df_train.Datetime = pd.to_datetime(df_train.Datetime)

df_train=pd.read_csv(train_path,index_col='Datetime',parse_dates=True)
df_train.drop(labels = 'ID', axis = 1)
df_train.head()

,ID,Item_ID,Category_3,Category_2,Category_1,Price,Number_Of_Sales
Datetime,,,,,,,
2014-01-01,30495_20140101,30495,0,2.0,90,165.123,1
2014-01-01,30375_20140101,30375,0,2.0,307,68.666,5
2014-01-01,30011_20140101,30011,0,3.0,67,253.314,2
2014-01-01,30864_20140101,30864,0,2.0,315,223.122,1
2014-01-01,30780_20140101,30780,1,2.0,132,28.750,1


In [76]:
df_train.columns

Index(['ID', 'Item_ID', 'Category_3', 'Category_2', 'Category_1', 'Price',
       'Number_Of_Sales'],
      dtype='object')

In [84]:
df_train.groupby(['Datetime','Item_ID'])['Price']

In [3]:
df = pd.read_csv(train_path)
df.Datetime = pd.to_datetime(df.Datetime)
df = df.set_index(['Item_ID','Datetime'])

In [4]:
df.head()

,,ID,Category_3,Category_2,Category_1,Price,Number_Of_Sales
Item_ID,Datetime,,,,,,
30495,2014-01-01,30495_20140101,0,2.0,90,165.123,1
30375,2014-01-01,30375_20140101,0,2.0,307,68.666,5
30011,2014-01-01,30011_20140101,0,3.0,67,253.314,2
30864,2014-01-01,30864_20140101,0,2.0,315,223.122,1
30780,2014-01-01,30780_20140101,1,2.0,132,28.750,1


In [5]:
df = df.sort_index(ascending=False)
df.sort_values(['Item_ID','Datetime'],ascending=[False,True],inplace=True)
df.reset_index(inplace=True)
df.head()

,Item_ID,Datetime,ID,Category_3,Category_2,Category_1,Price,Number_Of_Sales
0,31375,2015-05-27,31375_20150527,0,5.0,235,0.882,133
1,31375,2015-05-28,31375_20150528,0,5.0,235,0.787,113
2,31375,2015-05-29,31375_20150529,0,5.0,235,0.942,59
3,31375,2015-05-30,31375_20150530,0,5.0,235,1.049,56
4,31375,2015-05-31,31375_20150531,0,5.0,235,1.381,37


In [6]:
items = list(df.Item_ID.unique())

In [7]:
item1 = df[df.Item_ID == 30258]
item1.set_index(item1.Datetime,inplace=True)
item1.drop(labels=['index','ID'], axis = 1,inplace=True)
item1.head()

KeyError: "labels ['index'] not contained in axis"

In [27]:
item1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 885 entries, 2014-01-01 to 2016-06-03
Data columns (total 7 columns):
Item_ID            885 non-null int64
Datetime           885 non-null datetime64[ns]
Category_3         885 non-null int64
Category_2         885 non-null float64
Category_1         885 non-null int64
Price              885 non-null float64
Number_Of_Sales    885 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(4)
memory usage: 55.3 KB


In [39]:
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.8)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit(disp=0)
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	error = mean_squared_error(test, predictions)
	return error

evaluate_arima_model(item1['Price'], (1,1,1))

0.16785323314121522

In [40]:
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in tqdm(p_values):
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					mse = evaluate_arima_model(dataset, order)
					if mse < best_score:
						best_score, best_cfg = mse, order
					print('ARIMA%s MSE=%.3f' % (order,mse))
				except:
					continue
	print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))
    
p = [1,2]
q = [1]
d = [1]

evaluate_models(item1['Price'],p,d,q)

 50%|█████     | 1/2 [00:52<00:52, 52.61s/it]

ARIMA(1, 1, 1) MSE=0.168


100%|██████████| 2/2 [02:14<00:00, 67.42s/it]

ARIMA(2, 1, 1) MSE=0.168
Best ARIMA(2, 1, 1) MSE=0.168


In [8]:
import statsmodels.formula.api as sm

In [9]:
df.head()

,Item_ID,Datetime,ID,Category_3,Category_2,Category_1,Price,Number_Of_Sales
0,31375,2015-05-27,31375_20150527,0,5.0,235,0.882,133
1,31375,2015-05-28,31375_20150528,0,5.0,235,0.787,113
2,31375,2015-05-29,31375_20150529,0,5.0,235,0.942,59
3,31375,2015-05-30,31375_20150530,0,5.0,235,1.049,56
4,31375,2015-05-31,31375_20150531,0,5.0,235,1.381,37


In [ ]:
#FE_ols = sm.ols(formula='np.log(Price) ~ Datetime + Item_ID + C(Category_1) + C(Category_3)',data=df).fit()

In [10]:
from regressions.regressions import RDataFrame

In [11]:
dt = RDataFrame(df)

In [12]:
dt.xtset(i='Item_ID', t='Datetime')

In [13]:
dt.xtreg('Price ~ Item_ID + Datetime + Category_3', 'fe')

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     13.47
Date:                Sat, 02 Jun 2018   Prob (F-statistic):               0.00
Time:                        18:18:42   Log-Likelihood:            -2.8799e+06
No. Observations:              881876   AIC:                         5.762e+06
Df Residuals:                  879436   BIC:                         5.772e+06
Df Model:                         911                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

<bound method RegressionResults.summary of <statsmodels.regression.linear_model.OLSResults object at 0x7fd574798630>>

In [19]:
%%time
dt.xtreg('np.log(Price) ~ Category_1 - 1 ', 'fe', 'robust')

                            OLS Regression Results                            
Dep. Variable:          np.log(Price)   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 02 Jun 2018   Prob (F-statistic):                nan
Time:                        18:25:02   Log-Likelihood:            -5.6791e+05
No. Observations:              881876   AIC:                         1.136e+06
Df Residuals:                  880347   BIC:                         1.136e+06
Df Model:                           0                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Category_1    -0.0029   4.05e-17  -7.16e+13      0.0

<bound method RegressionResults.summary of <statsmodels.regression.linear_model.OLSResults object at 0x7fd5712b9d68>>